In [1]:
import random
random.seed(123)

# IC Design

In [6]:
import numpy as np
import pandas
np.random.seed(123)
# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.5
Le = 2.695

# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

# Simulating Phase I and Phase II
total_runs = []
for i in range(sim):
    zi_1 = exp_w  # Phase I mean as starting EWMA value
    xi = np.random.normal(0, 1, m)  # Phase I data
    count = 0
    signaled = False
    total_run = 0
    while not signaled:
        total_run += 1
        yi = np.random.normal(0, 1, n)  # Phase II data
        comb = np.concatenate((xi, yi))
        ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
        W = ranks[-n:].sum()  # Sum of ranks for new sample

        # Calculate W-EWMA
        zi = lambda_ * W + (1 - lambda_) * zi_1
        zi_1 = zi
        # Check control limits and evaluate run length
        if zi >= UCL or zi <= LCL:
            total_runs.append(total_run)
            signaled = True

run_lengths = np.array(total_runs)
print(f"Mean Run Length: {run_lengths.mean()}")
print(f"Standard Deviation: {run_lengths.std()}")
print(f"5th Percentile: {np.percentile(run_lengths, 5)}")
print(f"95th Percentile: {np.percentile(run_lengths, 95)}")
print(f"Median: {np.percentile(run_lengths, 50)}")


Mean Run Length: 201.2949
Standard Deviation: 258.25277294540325
5th Percentile: 10.0
95th Percentile: 697.0999999999985
Median: 115.0


## OOC

In [9]:
import numpy as np
import pandas as pd
import numpy
from tqdm import tqdm

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.05
Le = 2.5     # Lambda = 0.05
Le = 2.695   # Lambda = 0.5
df = 5
# df = 20
shape_param = 1  # Shape parameter for the gamma distribution
# shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution

lambda_values = [0.05, 0.5]
Le_values = [2.5, 2.695]

deltas = np.arange(0, 3.25, 0.25)
results = pandas.DataFrame(columns=['Delta', 'ARL (0.05)', 'SDRL (0.05)', 'MDRL (0.05)', 'ARL (0.5)', 'SDRL (0.5)', 'MDRL (0.5)'])

for lambda_, Le in zip(lambda_values, Le_values):
    LCL = exp_w - Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    UCL = exp_w + Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    arl_column = f'ARL ({lambda_})'
    sdrl_column = f'SDRL ({lambda_})'
    mdrl_column = f'MDRL ({lambda_})'
    for delta in deltas:
        print(delta)
        total_runs = []
        for i in tqdm(range(sim)):
            zi_1 = exp_w  # Phase I mean as starting EWMA value
            # xi = np.random.normal(0, 1, m)  # Phase I data Normal
            # xi = np.random.standard_t(df, m)  # Phase I data t
            xi = np.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
            xi=(xi-shape_param)/np.sqrt(shape_param)
            count = 0
            signaled = False
            total_run = 0
            while not signaled:
                total_run += 1
                # yi = np.random.normal(delta, 1, n)  # Phase II data
                # yi = np.random.standard_t(df, n) + delta  # Phase II data t
                y1 = np.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
                yi = (y1 - shape_param) / np.sqrt(shape_param) + (delta)
                comb = np.concatenate((xi, yi))
                ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
                W = ranks[-n:].sum()  # Sum of ranks for new sample
    
                # Calculate W-EWMA
                zi = lambda_ * W + (1 - lambda_) * zi_1
                zi_1 = zi
                # Check control limits and evaluate run length
                if zi >= UCL or zi <= LCL:
                    total_runs.append(total_run)
                    signaled = True
    
        run_lengths = np.array(total_runs)
        ARL = run_lengths.mean()
        SDRL = run_lengths.std()
        MDRL = np.percentile(run_lengths, 50)
    
        run_lengths = np.array(total_runs)
        ARL = run_lengths.mean()
        SDRL = run_lengths.std()
        MDRL = np.percentile(run_lengths, 50)

        if delta in results['Delta'].values:
            results.loc[results['Delta'] == delta, arl_column] = ARL
            results.loc[results['Delta'] == delta, sdrl_column] = SDRL
            results.loc[results['Delta'] == delta, mdrl_column] = MDRL
        else:
            new_row = pandas.DataFrame({'Delta': [delta], arl_column: [ARL], sdrl_column: [SDRL], mdrl_column: [MDRL]})
            results = pandas.concat([results, new_row], ignore_index=True)

results['ARL'] = results.apply(lambda row: f"({row['ARL (0.05)']:.2f}, {row['ARL (0.5)']:.2f})", axis=1)
results = results[['Delta', 'ARL']]
    
results


0.0


100%|██████████| 10000/10000 [01:18<00:00, 127.49it/s]


0.25


100%|██████████| 10000/10000 [00:06<00:00, 1597.52it/s]


0.5


...

2.75


100%|██████████| 10000/10000 [00:00<00:00, 21500.23it/s]


3.0


100%|██████████| 10000/10000 [00:00<00:00, 23773.65it/s]


,Delta,ARL
0,0.00,"(205.72, 201.53)"
1,0.25,"(19.26, 52.87)"
2,0.50,"(7.03, 7.81)"
3,0.75,"(5.02, 3.12)"
4,1.00,"(4.16, 2.19)"
5,1.25,"(3.70, 1.92)"
6,1.50,"(3.29, 1.72)"
7,1.75,"(3.07, 1.48)"
8,2.00,"(3.01, 1.23)"
9,2.25,"(3.00, 1.08)"
